## Phase portraits
In this Jupyter Notebook, we will solve systems of differential equations of the form dx/dt=f<sub>1</sub>(x,y), dy/dt=f<sub>2</sub>(x,y) graphically generating phase portraits. The next cell loads several modules needed for defining the right-hand side and for running the code. To proceed, click twice on the run button (the triangle in the top menubar) to run the current and the next cell.

In [3]:
%matplotlib widget
from math import *
from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve

toggle = False # if false, solve ODE; if true, find equilibria
labels = [0, 1]
    
def generate_phase_portrait(f, xrange, yrange, tmax):
    def rhs(t, x): return f(x[0], x[1])

    def plotdf(rhs, xrange, yrange, grid, ax):
        x = np.linspace(xrange[0], xrange[1], grid[0])
        y = np.linspace(yrange[0], yrange[1], grid[1])
        X, Y = np.meshgrid(x, y)
        DX, DY = rhs(X, Y)
        M = (np.hypot(DX, DY))
        M[M==0] = 1.0
        DX = DX/M
        DY = DY/M
        ax.quiver(X, Y, DX, DY, color='tab:green', angles='xy', alpha=0.5)

    def cross_top(t, x): return yrange[1]+0.5 - x[1]
    def cross_bot(t, x): return yrange[0]-0.5 - x[1]
    def cross_lef(t, x): return xrange[0]-0.5 - x[0]
    def cross_rig(t, x): return xrange[1]+0.5 - x[0]
    cross_top.terminal = True
    cross_bot.terminal = True
    cross_lef.terminal = True
    cross_rig.terminal = True
    
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    plt.subplots_adjust(bottom=0.2)
    button = plt.axes([0.35, 0.05, 0.36, 0.05])   # [left, bottom, width, height]
    labels[0] = button.text(0.05, 0.3, "Click here to find equilibria", fontsize=10)
    labels[1] = button.text(0.10, 0.3, "Click here to solve ODE", fontsize=10)
    labels[0].set_visible(True)
    labels[1].set_visible(False)
    button.set(xticks=[], yticks=[])
    button.set_facecolor('tab:green')

    ax.set_title("Phase portrait")
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_xlim(xrange)
    ax.set_ylim(yrange)
    ax.grid(True)
    plotdf(f, xrange, yrange, [11, 11], ax)
    plt.show()

    def solve_ode(t, x):
        solf = solve_ivp(rhs, [t,  tmax], x, t_eval=np.linspace(t,  tmax, 1000), 
                         events=[cross_top, cross_bot, cross_lef, cross_rig],
                         atol=1.e-8, rtol=1.e-6)
        solb = solve_ivp(rhs, [t, -tmax], x, t_eval=np.linspace(t, -tmax, 1000), 
                         events=[cross_top, cross_bot, cross_lef, cross_rig],
                         atol=1.e-8, rtol=1.e-6)
        ax.scatter(x[0], x[1], color='tab:olive', zorder=2)
        ax.plot(solf.y[0], solf.y[1], color='tab:blue', zorder=1)
        ax.plot(solb.y[0], solb.y[1], color='tab:blue', linestyle='dashed', zorder=1)

    def on_click(event):
        global toggle, labels
        if event.button is MouseButton.LEFT:
            if event.inaxes == button:
                labels[toggle].set_visible(False)
                toggle = not toggle
                labels[toggle].set_visible(True)
            if event.inaxes == ax:
                x = event.xdata
                y = event.ydata
                if x>xrange[0] and x<xrange[1] and y>yrange[0] and y<yrange[1]:
                    if toggle:
                        equilibrium = fsolve(lambda x: f(x[0], x[1]), [x, y])
                        ax.scatter(equilibrium[0], equilibrium[1], s=70, color='tab:red', zorder=2)
                    else: solve_ode(0, [x, y])
            fig.canvas.draw()

    plt.connect('button_press_event', on_click)

The next cell defines the right-hand side `f(x,y)` of our differential equation. Click again twice on the run button to execute this and the next cell. The code will create a direction field with arrows indicating the slope of the tangents of solution curves. If you position your mouse anywhere in the figure and click, the code will compute and plot the solution that starts at the location `(x,y)` of your mouse: solutions start at the grey points, solid curves correspond to t>0, and dashed curves to t<0. Try it out a few times to see what happens! You can also click on the green button on the bottom to switch to finding equilibria: once activated, click anywhere near a suspected equilibrium and the code will compute the equilibrium and plot it. Clicking again on the green button will switch back to computing solutions.

In [4]:
def f(x, y):
    return [y, -np.sin(x)]

generate_phase_portrait(f, xrange=[-4, 4], yrange=[-3.5, 3.5], tmax=20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

You can go back to the previous cell (click on the gray area containing the `def f(x,y):` statement) and edit the expression for f(x,y) or the values xrange, yrange, and tmax which define the regions in x and y that will be plotted and maximum value used for the time variable t that our solver will use. You can edit the expression for f that we use:
* Make sure that you use `**` for exponents (so `x**3` means x<sup>3</sup>). If you want to use trigonometric functions, use `np.sin(x)` or `np.cos(x)` for the sine or cosine functions.
* The code is written in Python. Python uses indentation to separate elements of the code, and you can see that the `return [x-x**2, -3*y]` statement in the definition of f(x,y) is indented: you need to leave this statement indented or the code will result in an error.

If your code no longer works, try to undo the last edits (see the "Edit" menu om the ipper left corner) in the cells in which you made changes or, if this fails, upload the original Jupyter notebook again.